Entrada: <posicionamento, alvo>

*Encontrar comentarios relacionados:*

*   **Filtragem:** comentarios = similaridade_semantica(corpus, alvo)

Detectar presença de posicionamento:

*   **Filtragem:** comentarios = detecção_posicionamento(comentarios, alvo)

Classificar posicionamento:

*   **Rotulação:** comentarios = classificação_posicionamento(comentarios, alvo, posicionamento)

Identificar principais tópicos citados:

*   **Identificar tópicos:** topicos_rel = modelagem_de_topicos(comentarios)

Ranquear resultados:

* **Ordenação:** comentarios = ranquear(alvo + topicos_rel, comentarios)


Exibição da saída:


---

## Comentários [posicionamento] à [tema]

---

Principais tópicos comentados: [topicos_gerados]


Top 2
*   Comentário 1
*   Comentário 2





# Similaridade Semantica

In [1]:
%%capture
!nvcc --version
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
%%capture
!pip install transformers

In [3]:
%%capture
!pip install -U sentence-transformers
!pip install faiss-gpu

In [4]:
%%capture
!pip install rank_bm25

In [5]:
%%capture
!pip install bertopic

In [6]:
%%capture
!pip install rapidfuzz

In [18]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.optim import Adam
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, TensorDataset, DataLoader
from transformers import BertTokenizer, BertModel
from transformers import BertTokenizer, TFBertModel
from transformers import BertForSequenceClassification, AdamW
from tqdm import tqdm
import pickle

from sklearn import metrics
from sklearn.preprocessing import MultiLabelBinarizer

from sentence_transformers import SentenceTransformer
import faiss

import os
import time

from rank_bm25 import BM25Okapi

import nltk
from nltk.corpus import stopwords
from bertopic import BERTopic
from rapidfuzz import process, fuzz

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
def gerar_embeddings_similaridade_semantica(df_data):
  t = time.time()
  model_msmarco_miniLM = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
  opinions=df_data["content"].tolist()
  opinions_embeddings=model_msmarco_miniLM.encode(opinions)
  index = faiss.IndexFlatL2(opinions_embeddings.shape[1])
  index.add(opinions_embeddings)
  faiss.write_index(index, 'index_opinions')
  index = faiss.read_index('index_opinions')
  print('totaltime gerar_embeddings_similaridade: {}m'.format((time.time()-t)/60))
  return index, opinions, model_msmarco_miniLM

def filtrar_comentarios_relacionados(alvo, cache, opinions, model, qtd=50) -> list:
  def search(query, topK, index):    
    t = time.time()
    query_vector = model.encode([query])
    k = topK
    top_k = index.search(query_vector, k)
    print('totaltime filtrar_comentarios: {}s'.format(time.time()-t))
    return [opinions[_id] for _id in top_k[1].tolist()[0]], top_k[1].tolist()[0]
  opinions_list, index_list = search(alvo, qtd, cache)
  return opinions_list, index_list


In [47]:
file_path = 'https://drive.google.com/u/1/uc?id=1KmFplvXLdJNrJydP-g2P6_VEeloAxKcJ&export=download&confirm=t'
df_raw_data = pd.read_csv(file_path, encoding='utf-8', sep=';')
df_raw_data[['Nome', 'Conteúdo']]

,Nome,Conteúdo
0,PL 11098/2018,Não entendi muito bem a proposta.
1,PL 7180/2014,Acabar com doutrinação nas escolas
2,PL 7180/2014,Doutrinar crianças
3,PL 7180/2014,Professores têm que exercer a liberdade de Cát...
4,PL 10996/2018,privatiza de forma simplista o setor de saneam...
...,...,...
215708,PL 2893/2019,"O aborto é feito legal ou clandestinamente, mu..."
215709,PL 4286/2020,Lei suja e perigosa. Milhões de homens ficaram...
215710,PL 2893/2019,A vítima de estupro é reiteradamente penalizad...
215711,PL 2893/2019,Usar casos de mulheres estupradas que OPTARAM ...


In [41]:
file_path = 'https://drive.google.com/u/1/uc?id=1NlXEYIc4wcZ_F60qNGttPRermwsmQTIZ&export=download&confirm=t'
df_corpus = pd.read_csv(file_path, encoding='utf-8')
df_corpus.columns =  ['_id', 'id', 'content']
df_corpus['content'] = df_corpus['content'].str.strip()
df_corpus = df_corpus.drop_duplicates(subset='content')

cache_index, cache_comentarios, modelo_similaridade = gerar_embeddings_similaridade_semantica(df_corpus)

totaltime gerar_embeddings_similaridade: 0.07454976638158163m


In [ ]:
#@title
# entrada = {'posicionamento': "Favoravel a", 'alvo': "aumento real salário mínimo"}

# comentarios, indices = filtrar_comentarios_relacionados(entrada['alvo'], cache_index,cache_comentarios, modelo_similaridade)
# # comentarios = filtrar_comentarios_semelhantes(entrada['alvo'], cache_index,cache_comentarios, modelo_similaridade)
# comentarios

In [10]:
class MNLIDataBert(Dataset):

  def __init__(self, train_df, val_df, label_dict={'against': 0, 'favor': 1}):
    self.label_dict = label_dict

    self.train_df = train_df
    self.val_df = val_df

    self.base_path = '/content/'
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
    self.train_data = None
    self.val_data = None
    self.init_data()

  def init_data(self):
    self.train_data = self.load_data(self.train_df)
    self.val_data = self.load_data(self.val_df)

  def load_data(self, df):
    MAX_LEN = 512
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['sentence1'].to_list()
    hypothesis_list = df['sentence2'].to_list()
    label_list = df['gold_label'].to_list()

    for (premise, hypothesis, label) in zip(premise_list, hypothesis_list, label_list):
      sentence1_id = self.tokenizer.encode(premise, add_special_tokens = False)
      sentence2_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
      pair_token_ids = [self.tokenizer.cls_token_id] + sentence1_id + [self.tokenizer.sep_token_id] + sentence2_id + [self.tokenizer.sep_token_id]
      sentence1_len = len(sentence1_id)
      sentence2_len = len(sentence2_id)

      segment_ids = torch.tensor([0] * (sentence1_len + 2) + [1] * (sentence2_len + 1)) 
      attention_mask_ids = torch.tensor([1] * (sentence1_len + sentence2_len + 3))

      token_ids.append(torch.tensor(pair_token_ids))
      seg_ids.append(segment_ids)
      mask_ids.append(attention_mask_ids)
      y.append(self.label_dict[label])
    
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)
    y = torch.tensor(y)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids, y)
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    train_loader = DataLoader(
      self.train_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    val_loader = DataLoader(
      self.val_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    return train_loader, val_loader


def evaluateSD(y_test, model, deflabels={'against': 0, 'favor': 1}):    
  outputs = []
  outputs_prob = []
  test_labels = []
  mnli_dataset_ = MNLIDataBert(y_test, y_test, deflabels)
  test_dataloader, _ = mnli_dataset_.get_data_loaders(batch_size=1)
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  if use_cuda:
      model_ = model.cuda()
  
  with torch.no_grad():
      for pair_token_ids, mask_ids, seg_ids, y in test_dataloader:
            pair_token_ids = pair_token_ids.to(device)
            mask_ids = mask_ids.to(device)
            seg_ids = seg_ids.to(device)
            
            labels = y.to(device)
            output = model_(pair_token_ids, mask_ids, seg_ids)
            prob = output.logits

            outputs_prob.append(tuple(prob.cpu().detach().numpy()))

            outputs.append(tuple(output.logits.argmax(dim=1).cpu().detach().numpy()))
            test_labels.append(tuple(y.cpu().detach().numpy()))

  predicted, predicted_prob, test_labels = outputs, outputs_prob, test_labels
  return predicted, predicted_prob, test_labels


In [28]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
modelBERTSD = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2, )

def rotular_posicionamento(comentarios, alvo, posicionamento_filtro):
  t = time.time()
  modelBERTSD.load_state_dict(torch.load('/content/drive/MyDrive/bertmodels/sent_entail_bert_sd_classify.pt'))
  modelBERTSD.to(device)

  df_data = pd.DataFrame(comentarios, columns=['sentence2'])
  df_data['gold_label'] = ['favor']*len(df_data)
  df_data['sentence1'] = [alvo]*len(df_data)
  outputs, _, _ = evaluateSD(df_data, modelBERTSD, deflabels={'against': 0, 'favor': 1})
  df_data['predito'] = outputs
  print('totaltime rotular_comentarios: {}s'.format(time.time()-t), posicionamento_filtro)
  # print(df_data['predito'], len(df_data),  len(df_data[df_data['predito'] == posicionamento_filtro]['sentence2']))
  return df_data[df_data['predito'] == posicionamento_filtro]['sentence2']

def identificar_presenca_posicionamento(comentarios, alvo, posicionamento_filtro):
  t = time.time()
  modelBERTSD.load_state_dict(torch.load('/content/drive/MyDrive/bertmodels/sent_entail_bert_sd_identify.pt'))
  modelBERTSD.to(device)

  df_data = pd.DataFrame(comentarios, columns=['sentence2'])
  df_data['gold_label'] = ['stance']*len(df_data)
  df_data['sentence1'] = [alvo]*len(df_data)
  outputs, _, _ = evaluateSD(df_data, modelBERTSD, deflabels={'stance': 0, 'neither': 1})
  df_data['predito'] = outputs
  print('totaltime filtrar_comentarios: {}s'.format(time.time()-t))
  return df_data[df_data['predito'] == posicionamento_filtro]['sentence2']

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
#@title
# comentarios_rotulo = rotular_posicionamento(comentarios, posicionamento_filtro=(1,)).values

In [44]:
nltk.download('stopwords')
stopwords_pt = stopwords.words("portuguese")

extras = ['são','sim','nao','não','ser','bem','pode', 'vai', 'fim', 'todos', 'todo', 'nenhum', 'quero', 'irá', 'cada', 'tudo', 'porque', 'pl', 'muitas', '1']
with open("/content/drive/MyDrive/mestrado/3 sem/NLP Projeto/stop_words.txt", 'r') as arquivo:
  for linha in arquivo:
    for palavra in linha.split():
      extras.append(palavra)

for palavra in extras:
  stopwords_pt.append(palavra)

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


def extrair_topicos(coment):
  docs = [' '.join([a for a in x.split() if a not in stopwords_pt]) for x in coment]
  topic_model = BERTopic(language="multilingual")
  topics, probs = topic_model.fit_transform(docs)
  info_topics = topic_model.get_topic_info()
  topicos = []
  if len(info_topics.Name) > 0:
    topicos = info_topics.Name[0].split('_')
  return topicos[1:]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#@title
# topicos = extrair_topicos(comentarios)

In [15]:
def ranquear(coment, alvo):
  tokenized_corpus = [doc.split(" ") for doc in coment]
  bm25 = BM25Okapi(tokenized_corpus)
  query = alvo
  tokenized_query = query.split(" ")
  doc_scores = bm25.get_scores(tokenized_query)
  return bm25.get_top_n(tokenized_query, coment, n=10)

In [ ]:
#@title
# ranqueados = ranquear(comentarios_rotulo, entrada['alvo']+' '+' '.join(topicos))

In [ ]:
#@title
# from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
# text = "An ambitious campus expansion plan was proposed by Fr. Vernon F. Gallagher in 1952. Assumption Hall, the first student dormitory, was opened in 1954, and Rockwell Hall was dedicated in November 1958, housing the schools of business and law. It was during the tenure of F. Henry J. McAnulty that Fr. Gallagher's ambitious plans were put to action."

# # Training a new model with the text.
# tokenizer = PunktSentenceTokenizer()
# tokenizer.train(comentarios)

# # It automatically learns the abbreviations.
# tokenizer._params.abbrev_types

# # Use the customized tokenizer.
# tokenizer.tokenize(comentarios[1])

In [ ]:
#@title
# segmentado = []
# for i in ranqueados:
#   segmentado.append(tokenizer.tokenize(i))

In [ ]:
#@title
# segmentado[0]

# for j in segmentado:
#   j

In [ ]:
#@title
# for j in segmentado:
#   j = pd.DataFrame(j, columns=['content'])
#   cache_index, cache_comentarios, modelo_similaridade = gerar_embeddings_similaridade_semantica(j)
#   comentarios_, indices = filtrar_comentarios_relacionados(entrada['alvo']+' '+' '.join(topicos), cache_index,cache_comentarios, modelo_similaridade)
#   print(comentarios_)
#   print(j)

In [54]:
def pipeline(posicionamento, topico):
  pos_dict = {"favoraveis": 1, "contrarios": 0}
  # entrada = {'posicionamento': pos_dict[posicionamento], 'alvo': topico}
  try:
    comentarios__, indices__ = filtrar_comentarios_relacionados(topico, cache_index,cache_comentarios, modelo_similaridade, qtd=300)
    comentarios__ = process.extract(topico, comentarios__, scorer=fuzz.WRatio, limit=150)
    comentarios__ = [i[0] for i in comentarios__]
    comentarios__ = identificar_presenca_posicionamento(comentarios__, topico, (0,)).values
    comentarios_rotulados = rotular_posicionamento(comentarios__, topico, posicionamento_filtro=(pos_dict[posicionamento],)).values
    try:
      topicos = extrair_topicos(comentarios_rotulados)
    except:
      topicos = []
    ranqueados = ranquear(comentarios_rotulados, topico)
  except:
    topicos, ranqueados = [],[]
  return topicos, ranqueados

In [ ]:
#@title
# import jellyfish
# print(jellyfish.levenshtein_distance('servidor público', 'A proposta lamentavelmente afeta a estabilidade e a integridade dos cargos públicos, ensejando indesejável ingerência política na atuação dos servidores, prejudicando a prestação regular dos serviços públicos.'))

# print(jellyfish.jaro_distance('armas', 'A proposta lamentavelmente afeta a estabilidade e a integridade dos cargos públicos, ensejando indesejável ingerência política na atuação dos servidores, prejudicando a prestação regular dos serviços públicos.'))

# print(jellyfish.damerau_levenshtein_distance('servidor público', 'A proposta lamentavelmente afeta a estabilidade e a integridade dos cargos públicos, ensejando indesejável ingerência política na atuação dos servidores, prejudicando a prestação regular dos serviços públicos.'))

# from rapidfuzz import process, fuzz
# comentarios__, indices__ = filtrar_comentarios_relacionados("servidor público", cache_index,cache_comentarios, modelo_similaridade, qtd=200)
# comentarios__ = process.extract("servidor público", comentarios__, scorer=fuzz.WRatio, limit=200)
# [i[0] for i in comentarios__]

In [ ]:
# coding=utf8
#@markdown ---
#@markdown ### Buscar comentários:
posicionamento = "contrarios" #@param ["favoraveis", "contrarios"]
topico = "estatuto do desarmamento" #@param {type:"string"}
topico_ = topico.encode().decode('utf-8')

import IPython

topicos_gerados, ranqueados = pipeline(posicionamento, topico_)

texto = f'''
---

## Comentários *{posicionamento}* à *{topico_}*

---

Principais tópicos comentados: *{', '.join(topicos_gerados)}*


'''
comentarios = ranqueados[:20]
if len(comentarios) > 0:
  texto += f'Top {len(comentarios)} Comentários:\n\n'
  for comentario in comentarios:
    texto += f'*   {comentario}\n'
else:
  texto += 'Não há comentários'

IPython.display.Markdown(texto) 


totaltime filtrar_comentarios: 0.0582585334777832s
200
200
totaltime rotular_comentarios: 7.37035346031189s (0,)
0      (0,)
1      (0,)
2      (0,)
3      (0,)
4      (0,)
       ... 
195    (0,)
196    (0,)
197    (0,)
198    (0,)
199    (0,)
Name: predito, Length: 200, dtype: object 200 186
10



---

## Comentários *contrarios* à *estatuto do desarmamento*

---

Principais tópicos comentados: *reforma, ideologica, colocar, conhecimento *


Top 10 Comentários:

*   População quer a revogação do estatuto do desarmamento
*   retira a subjetividade do estatuto do desarmamento.
*   respeitem o referendo e o estatuto do desarmamento
*   É um direito do cidadão, e o estatuto do desarmamento só piorou não deu resultado.
*   Não  existe nenhum, não cumpriram a vontade do povo decente contra o estatuto do desarmamento é inconstitucional , 65% dos brasileiros VOTARAM contra o estatuto do desarmamento no plebiscito de 2005. VOCÊS devem CUMPRIR
*   Ridículo, interesse em manipular e controlar a sociedade, semelhante o governo do PT com estatuto do desarmamento
*   Não respeitaram mais uma vez o resultado do referendo sobre o estatuto do desarmamento.
*   O estatuto do desarmamento retira o direito a proteção a vida. .
*   Garantir direito já previsto no estatuto do desarmamento lei 13.826/2003 ART 10
*   O estatuto do desarmamento não funcionou, quero viver como antes de 2004


In [30]:
# coding=utf8
#@markdown ---
#@markdown ### Buscar comentários:
posicionamento = "favoraveis" #@param ["favoraveis", "contrarios"]
topico = "estatuto do desarmamento" #@param {type:"string"}
topico_ = topico.encode().decode('utf-8')

import IPython

topicos_gerados, ranqueados = pipeline(posicionamento, topico_)

texto = f'''
---

## Comentários *{posicionamento}* à *{topico_}*

---

Principais tópicos comentados: *{', '.join(topicos_gerados)}*


'''
comentarios = ranqueados[:20]
if len(comentarios) > 0:
  texto += f'Top {len(comentarios)} Comentários:\n\n'
  for comentario in comentarios:
    texto += f'*   {comentario}\n'
else:
  texto += 'Não há comentários'

IPython.display.Markdown(texto) 


totaltime filtrar_comentarios: 0.020627260208129883s
totaltime filtrar_comentarios: 4.408422231674194s
totaltime rotular_comentarios: 3.637830972671509s (1,)



---

## Comentários *favoraveis* à *estatuto do desarmamento*

---

Principais tópicos comentados: **


Não há comentários

In [31]:
# coding=utf8
#@markdown ---
#@markdown ### Buscar comentários:
posicionamento = "favoraveis" #@param ["favoraveis", "contrarios"]
topico = "Servidor p\xFAblico" #@param {type:"string"}
topico_ = topico.encode().decode('utf-8')

import IPython

topicos_gerados, ranqueados = pipeline(posicionamento, topico_)

texto = f'''
---

## Comentários *{posicionamento}* à *{topico_}*

---

Principais tópicos comentados: *{', '.join(topicos_gerados)}*


'''
comentarios = ranqueados[:20]
if len(comentarios) > 0:
  texto += f'Top {len(comentarios)} Comentários:\n\n'
  for comentario in comentarios:
    texto += f'*   {comentario}\n'
else:
  texto += 'Não há comentários'

IPython.display.Markdown(texto) 

totaltime filtrar_comentarios: 0.020840883255004883s
totaltime filtrar_comentarios: 4.082974195480347s
totaltime rotular_comentarios: 3.3310084342956543s (1,)



---

## Comentários *favoraveis* à *Servidor público*

---

Principais tópicos comentados: *públicos, servidores, servidor, estabilidade*


Top 10 Comentários:

*   O fim da estabilidade é uma ponte para contratação de pessoas sem qualificação . Servidor publico deve prestar concurso publico e ter estabilidade sim, esse é o prêmio por ser o melhor entre tantos
*   Desmonte do serviço público para abrir espaço para contratação terceirizada de forma duvidosa
*   Supressão de salário dos servidores públicos é retirar de hipossuficiente. Peguem dos bancos, das grandes fortunas, e não do trabalhador servidor público
*   Bom servidor com competência poderá ter inclusive mais de um cargo. Contratação por CLT garante direitos que servidor público atualmente não tem, como FGTS
*   O servidor público já foi severamente penalizado na reforma previdenciária. Se é pra cortar salários deve-se começar pelos verdadeiros privilegiados do país: políticos, juízes e apadrinhados.
*   Fere a ordem constitucional e, antes de tudo, a moral! Contratação pelo poder público é só por concurso! Sobretudo, a lei deve ser genérica e abstrata!
*   Irá ajudar principalmente o servidor público com salário menor, pois o consignado é a contratação mais utilizada e nessa pandemia há registro de corte em alguns benefícios.
*   Se há necessidade de uma reforma administrativa, deve-se abarcar TODOS que prestam serviço público, desde os altos escalões, sem restringir apenas o servidor público concursado das carreiras iniciais.
*   A PEC vai regredir diversos anos em relação à qualidade nada contratações, com a possibilidade de contratação sem concursos públicos. Voltaremos à época em que o serviço público servia de cabide eleitoral. .
*   A perda da estabilidade, que protege o servidor público a desenvolver seu trabalho. A possibilidade de contratação sem concurso público. Tira o mérito das pessoas que estudam para conseguirem emprego através de seus esforços..


In [36]:
# coding=utf8
#@markdown ---
#@markdown ### Buscar comentários:
posicionamento = "contrarios" #@param ["favoraveis", "contrarios"]
topico = "Servidor p\xFAblico" #@param {type:"string"}
topico_ = topico.encode().decode('utf-8')

import IPython

topicos_gerados, ranqueados = pipeline(posicionamento, topico_)

texto = f'''
---

## Comentários *{posicionamento}* à *{topico_}*

---

Principais tópicos comentados: *{', '.join(topicos_gerados)}*


'''
comentarios = ranqueados[:20]
if len(comentarios) > 0:
  texto += f'Top {len(comentarios)} Comentários:\n\n'
  for comentario in comentarios:
    texto += f'*   {comentario}\n'
else:
  texto += 'Não há comentários'

IPython.display.Markdown(texto) 

totaltime filtrar_comentarios: 0.019636154174804688s
totaltime filtrar_comentarios: 4.08664083480835s
totaltime rotular_comentarios: 3.2774112224578857s (0,)



---

## Comentários *contrarios* à *Servidor público*

---

Principais tópicos comentados: **


Não há comentários

In [52]:
# coding=utf8
#@markdown ---
#@markdown ### Buscar comentários:
posicionamento = "contrarios" #@param ["favoraveis", "contrarios"]
topico = "Reforma Previdenci\xE1ria" #@param {type:"string"}
topico_ = topico.encode().decode('utf-8')

import IPython

topicos_gerados, ranqueados = pipeline(posicionamento, topico_)

texto = f'''
---

## Comentários *{posicionamento}* à *{topico_}*

---

Principais tópicos comentados: *{', '.join(topicos_gerados)}*


'''
comentarios = ranqueados[:20]
if len(comentarios) > 0:
  texto += f'Top {len(comentarios)} Comentários:\n\n'
  for comentario in comentarios:
    texto += f'*   {comentario}\n'
else:
  texto += 'Não há comentários'

IPython.display.Markdown(texto) 

totaltime filtrar_comentarios: 0.021097421646118164s
totaltime filtrar_comentarios: 4.141027212142944s
totaltime rotular_comentarios: 3.9695630073547363s (0,)
0      (0,)
1      (0,)
2      (0,)
3      (0,)
5      (0,)
       ... 
139    (0,)
140    (0,)
141    (0,)
143    (0,)
144    (0,)
Name: predito, Length: 135, dtype: object



---

## Comentários *contrarios* à *Reforma Previdenciária*

---

Principais tópicos comentados: *reforma, tributária, contribuição, previdenciária*


Top 10 Comentários:

*   Retirada de direitos dos trabalhadores. Acabaram de votar a Reforma Previdenciária e o governo já promove isenção de tributos aos empresários com cobrança de tributo sobre seguro-desemprego. Um absurdo!
*   Incluirá os Estados e Municípios na Reforma Previdenciária e fará justiça aos que estavam na expectativa de aposentar nos próximos dois anos e foram prejudicados com a mudança na forma dos cálculos dos benefícios.
*   A Nova Previdência nos traz segurança econômica e garantia de que nós e as futuras gerações continuaremos recebendo aposentadoria em dia. Eu confio no Presidente Bolsonaro e apoio a Reforma Previdenciária proposta pelo Governo.
*   A Nova Previdência Social não estabelece uma idade mínima única para todas as carreiras do Funcionalismo de Estado. Essa Reforma Previdenciária deveria cobrar uns 15% de contribuição para constituição do Pecúlio, a reserva de dinheiro da aposentadoria dos Servidores Públicos.
*   Reforma Política e Reforma Tributária parada.
*   Reforma tributária primeiro.
*   Tem pautas muito mais importantes para trabalhar.. Não ao fundo partidária.. Reforma tributária.. Lei anti corrupção.. Reforma trabalhista para todos.
*   A redução  do benefício  do Loas.... Dentro da Reforma  da Previdência  está  atrelada assuntos  relativos a Reforma  Trabalhista....
*   Reforma política e reforma tributária em primeiro lugar!!
*   Relevância 0. Reforma tributária e reforma política, depois o resto.


In [58]:
# coding=utf8
#@markdown ---
#@markdown ### Buscar comentários:
posicionamento = "favoraveis" #@param ["favoraveis", "contrarios"]
topico = "Reforma Previdenci\xE1ria" #@param {type:"string"}
topico_ = topico.encode().decode('utf-8')

import IPython

topicos_gerados, ranqueados = pipeline(posicionamento, topico_)

texto = f'''
---

## Comentários *{posicionamento}* à *{topico_}*

---

Principais tópicos comentados: *{', '.join(topicos_gerados)}*


'''
comentarios = ranqueados[:20]
if len(comentarios) > 0:
  texto += f'Top {len(comentarios)} Comentários:\n\n'
  for comentario in comentarios:
    texto += f'*   {comentario}\n'
else:
  texto += 'Não há comentários'

IPython.display.Markdown(texto) 

totaltime filtrar_comentarios: 0.021738529205322266s
totaltime filtrar_comentarios: 4.114678382873535s
totaltime rotular_comentarios: 3.9707272052764893s (1,)



---

## Comentários *favoraveis* à *Reforma Previdenciária*

---

Principais tópicos comentados: *reforma, tributária, previdência, problema*


Top 10 Comentários:

*   Reforma trabalhista vai gerar empregos - Não Gerou. Reforma da Previdencia vai salvar a Economia - Já admitem que não sera suficiente. Reduzir Salário de servidores Públicos é a solução - .... Cortar verba indenizatória, auxilio moradia e auxilio paletó ninguém quer né.
*   Haverá menos arrecadação, uma vez que a maioria da população não terá condições de contribuírem com a Previdência devido a informalidade laboral da maioria dos brasileiros. Uma Reforma Tributária  alcançará o equilíbrio econômico almejado.
*   A reforma atende ao mercado e aos bancos. O problema da economia resolva com a Reforma Tributária, pois pagamos muitos impostos. O governo faça gestão com os recursos públicos, pois dinheiro tem, o que falta é melhor gestão dos governantes.
*   Querer diminuir o prazo prescricional trabalhista que já é baixo, se comparado ao da lei Civil? ABSURDO. Os princípios do direito do trabalho, são protecionistas justamente pq não se trata de uma relação entre iguais. A reforma não criou postos de trabalho. .
*   Perda de direitos e garantia de sobrevivência na velhice. Como sempre é o mais pobre pagando pelos desmandos dos mais ricos. Chega de engodo!!! A reforma não vai gerar mais emprego, assim como a reforma trabalhista não gerou.
*   Essa reforma é PÉSSIMA para os trabalhadores. O problema não esta na PREVIDÊNCIA e sim nos JUROS DA DÍVIDA PÚBLICA. A reforma tem que ser a TRIBUTÁRIA. Bancos ganham com os juros da dívida pública. O problema esta nisso e não na PREVIDÊNCIA.
*   Cobre primeiro as empresas que devem muito, cortem regalias dos políticos e façam a reforma tributária. Não faz sentido tira o pouco que o trabalhador tem.
*   REFORMA É DESUMANA COM OS MAIS POBRES. COBREM AS DÍVIDAS DAS GRANDES EMPRESAS. FAÇAM A REFORMA TRIBUTÁRIA.
*   Pune os mais pobres! E também acho que deveria fazer uma reforma tributária antes de pensar em reformada previdência! É preciso taxar grandes fortunas!
*   Uma reforma previdenciária é necessário, mais para ampliar os direitos e não para diminuí-los ou cessá-los. Esta reforma como está é assassina.


In [59]:
# coding=utf8
#@markdown ---
#@markdown ### Buscar comentários:
posicionamento = "favoraveis" #@param ["favoraveis", "contrarios"]
topico = "aumento real sal\xE1rio m\xEDnimo" #@param {type:"string"}
topico_ = topico.encode().decode('utf-8')

import IPython

topicos_gerados, ranqueados = pipeline(posicionamento, topico_)

texto = f'''
---

## Comentários *{posicionamento}* à *{topico_}*

---

Principais tópicos comentados: *{', '.join(topicos_gerados)}*


'''
comentarios = ranqueados[:20]
if len(comentarios) > 0:
  texto += f'Top {len(comentarios)} Comentários:\n\n'
  for comentario in comentarios:
    texto += f'*   {comentario}\n'
else:
  texto += 'Não há comentários'

IPython.display.Markdown(texto) 

totaltime filtrar_comentarios: 0.021773338317871094s
totaltime filtrar_comentarios: 4.199463605880737s
totaltime rotular_comentarios: 3.2777209281921387s (1,)



---

## Comentários *favoraveis* à *aumento real salário mínimo*

---

Principais tópicos comentados: *aumento, salário, reforma, salarial*


Top 10 Comentários:

*   Nenhum ponto positivo nessa lei. A inflação real no Brasil não segue nenhum índice oficial do governo e o reajuste pela inflação deveria manter o poder de compra, mas isso não ocorre. O salário mínimo tenta fazer isso e assim é justo ser usado como vínculo de outros salários.
*   desatrelando, no entanto, do salário mínimo o valor pago para os benefícios assistenciais (LOAS – Lei Orgânica de Assistência Social), que independem de contribuição e que são devidos para pessoas carentes
*   Todas as empresas tenta baixar o valor do salário dos engenheiros trocando em seu cadastro o nome engenheiro por analista, gerente de obra, profissional de nível superior etc. Derrubar o salário mínimo da classe é corroborar com empresas fora da lei.
*   Bom fico feliz por alguém finalmente fazer uma lei que vai nos beneficiar agora falta fazer uma que almente o salário mínimo e tbm  permita que o bpc se torne em aposentadoria e não um benefício.
*   Acredito que a revogação na íntegra da lei seja um equívoco, mas as questões de salário mínimo para Engenheiros Júnior's com certeza deve ser revista, para que estes possam entrar no mercado de trabalho como tal e não como analistas ou qualquer outra denominação de cargo.
*   Regras de transição para o servidor público e para CLT.. Regras de aposentadoria familiar por atividade rural.. Regar do BPC passar para receber o salaio mínimo só a partir de 70 anos. .
*   Devido o fato de haver oportunidades de emprego, e ao mesmo tempo o número de demissões continuam ascendentes. é impressíndível que os assegurados sejam beneficiados com, no mínimo mais uma parcela do seguro desemprego.
*   É inconstitucional essa redução. Não recebo nenhuma ajuda de custo ou benefício do governo.  Não é justo querer amenizar um problema que é desigualdade social, causando outro. As contas não deixaram de vir ou serão reduzida porque o meu salário foi. Meus filhos terão que ser privado de um mínimo de dignidade pois não terei como arcar com minhas despesas. Não concordo.
*   Retrocessos nos direitos trabalhistas que beneficia apenas o empresariado. Possibilidade ainda de aumento no desemprego pelo aumento da jornada, retira do a necessidade de contratação.
*   o aumento de carga horária proposto deveria vir junto com aumento do números de turnos para dois ou mais e contratação urgente de mais funcionáros.


In [61]:
# coding=utf8
#@markdown ---
#@markdown ### Buscar comentários:
posicionamento = "contrarios" #@param ["favoraveis", "contrarios"]
topico = "C\xF3digo de Defesa do Consumidor" #@param {type:"string"}
topico_ = topico.encode().decode('utf-8')

import IPython

topicos_gerados, ranqueados = pipeline(posicionamento, topico_)

texto = f'''
---

## Comentários *{posicionamento}* à *{topico_}*

---

Principais tópicos comentados: *{', '.join(topicos_gerados)}*


'''
comentarios = ranqueados[:20]
if len(comentarios) > 0:
  texto += f'Top {len(comentarios)} Comentários:\n\n'
  for comentario in comentarios:
    texto += f'*   {comentario}\n'
else:
  texto += 'Não há comentários'

IPython.display.Markdown(texto) 

totaltime filtrar_comentarios: 0.021612167358398438s
totaltime filtrar_comentarios: 4.125352382659912s
totaltime rotular_comentarios: 2.269434690475464s (0,)


/usr/local/lib/python3.8/dist-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1593: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "
/usr/local/lib/python3.8/dist-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1593: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "



---

## Comentários *contrarios* à *Código de Defesa do Consumidor*

---

Principais tópicos comentados: **


Top 5 Comentários:

*   O ponto mais importante do Código de Trânsito Brasileiro é a segurança.. O trânsito vai ficar menos seguro com a aprovação desse PL
*   ridiculo fazer o beneficiario do BPC LOAS DEFICIENTE PASSAR POR PERICIA.TINHA E QUE APOSENTAR O MESMO.
*   Sinto falta de dispositivos constantes na Lei nº 12.846/2013 – Lei Anticorrupção – fazendo parte da cadeia logística criminosa empresas formalmente constituídas, estas estariam passivas de punições na esfera administrativa e judicial.
*   A economia com essa reforma será mínima, primeiro pq haverá contratação, e sabe se que o custo não é baixo, hj um posto terceirizado de vigilante custa para o Estado acima de 2 dígitos.
*   A criminalização de qualquer substância apenas beneficia aqueles que estão dispostos a comercializá-la ilegalmente (vide lei seca). Este projeto se esgota ainda no uso medicinal da planta. Não vejo, portanto, qual seria o problema.


In [56]:
#@title
def rotular_posicionamento(comentarios, alvo, posicionamento_filtro):
  t = time.time()
  modelBERTSD.load_state_dict(torch.load('/content/drive/MyDrive/bertmodels/sent_entail_bert_sd_classify.pt'))
  modelBERTSD.to(device)

  df_data = pd.DataFrame(comentarios, columns=['sentence2'])
  df_data['gold_label'] = ['favor']*len(df_data)
  df_data['sentence1'] = [alvo]*len(df_data)
  outputs, _, _ = evaluateSD(df_data, modelBERTSD, deflabels={'against': 0, 'favor': 1})
  df_data['predito'] = outputs
  print('totaltime rotular_comentarios: {}s'.format(time.time()-t), posicionamento_filtro)
  # print(df_data['predito'], len(df_data),  len(df_data[df_data['predito'] == posicionamento_filtro]['sentence2']))
  a = df_data[df_data['predito'] == posicionamento_filtro]
  # print(a['predito'])
  return a['sentence2']

In [49]:
#@title
def pipeline(posicionamento, topico):
  pos_dict = {"favoraveis": 1, "contrarios": 0}
  comentarios__, indices__ = filtrar_comentarios_relacionados(topico, cache_index,cache_comentarios, modelo_similaridade, qtd=300)
  comentarios__ = process.extract(topico, comentarios__, scorer=fuzz.WRatio, limit=150)
  comentarios__ = [i[0] for i in comentarios__]
  comentarios__ = identificar_presenca_posicionamento(comentarios__, topico, (0,)).values
  comentarios_rotulados = rotular_posicionamento(comentarios__, topico, posicionamento_filtro=(pos_dict[posicionamento],)).values
  topicos = extrair_topicos(comentarios_rotulados)
  ranqueados = ranquear(comentarios_rotulados, topico)
  return topicos, ranqueados